In [ ]:
#Downloading translated to Keras model and wights (original model is implemented on C and Darknet)
#https://drive.google.com/file/d/1--xywTuz4928rVzLP7sbBL3lH-xpJR9G/view?usp=sharing

!gdown --id 1--xywTuz4928rVzLP7sbBL3lH-xpJR9G


In [ ]:
!unzip YOLOv3.zip 

In [ ]:
#uplaoding some test images to the environment
#https://drive.google.com/file/d/1A9e2tZ-EP7oLH1FU_KRfE7GXTS3-C40y/view?usp=sharing
#https://drive.google.com/file/d/1TbzCbx-dQTxG67duFJHxAwq-sEQyPARm/view?usp=sharing
#https://drive.google.com/file/d/1pCjy-GjGowyeE9KWe2_QHh2OMrkHnAMD/view?usp=sharing
!gdown --id 1pCjy-GjGowyeE9KWe2_QHh2OMrkHnAMD
!gdown --id 1TbzCbx-dQTxG67duFJHxAwq-sEQyPARm
!gdown --id 1A9e2tZ-EP7oLH1FU_KRfE7GXTS3-C40y


In [ ]:
#copy the packages and images to the content folder
!cp -r 'content/YOLOv3/.' /content
!cp test.jpg test2.jpg test3.jpg /content/images

In [ ]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
from PIL import ImageFont, ImageDraw
import PIL.Image
import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor
import time

#model packages
from tensorflow.keras.models import load_model
from yad2k.models.keras_yolo import yolo_head
from yad2k.utils.utils import draw_boxes, get_colors_for_classes, scale_boxes, read_classes, read_anchors, preprocess_image
%matplotlib inline

In [ ]:
# utils

#changing box coordinate to corners (easier for processing)
def yolo_boxes_to_corners(box_xy, box_wh):
    """Convert YOLO box predictions to bounding box corners."""
    box_mins = box_xy - (box_wh / 2.)
    box_maxes = box_xy + (box_wh / 2.)

    return tf.keras.backend.concatenate([
        box_mins[..., 1:2],  # y_min
        box_mins[..., 0:1],  # x_min
        box_maxes[..., 1:2],  # y_max
        box_maxes[..., 0:1]  # x_max
    ])

In [ ]:
#filtering boxes based on score/proba
def yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = .6):
    x = 10
    y = tf.constant(100)
    # Compute box scores
    box_scores = box_confidence * box_class_probs

    # Find the box_classes using the max box_scores, keep track of the corresponding score
    box_classes      = tf.math.argmax(box_scores, axis=-1)
    box_class_scores = tf.math.reduce_max(box_scores, axis=-1, keepdims=False)
    
    # Create a filtering mask based on "box_class_scores" by using "threshold"(with probability >= threshold)
    filtering_mask = (box_class_scores >= threshold)
    
    # Applying the mask to box_class_scores, boxes and box_classes
    scores = tf.boolean_mask( box_class_scores, filtering_mask)
    boxes = tf.boolean_mask( boxes, filtering_mask)
    classes = tf.boolean_mask( box_classes, filtering_mask)

    return scores, boxes, classes

In [ ]:
#non max suppression algorithm based on IoU (intersection over union)
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    
    max_boxes_tensor = tf.Variable(max_boxes, dtype='int32')     # tensor to be used in tf.image.non_max_suppression()
    
    # Use tf.image.non_max_suppression() to get the list of indices corresponding to boxes you keep
    nms_indices =tf.image.non_max_suppression(boxes, scores, max_boxes, iou_threshold=iou_threshold, score_threshold=float('-inf'))
     
    #tf.gather() to select only nms_indices from scores, boxes and classes
    
    scores = tf.gather(scores, nms_indices)
    boxes = tf.gather(boxes, nms_indices)
    classes = tf.gather(classes, nms_indices)
    
    return scores, boxes, classes

In [ ]:
#function that process the raw output of the YOLOv3 model
def yolo_eval(yolo_outputs, image_shape = (720, 1280), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    
    # Retrieve outputs of the YOLO model 
    box_xy, box_wh, box_confidence, box_class_probs = yolo_outputs

    # Convert boxes to be ready for filtering functions
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    #Score-filtering with a threshold of score_threshold
    scores, boxes, classes = yolo_filter_boxes(boxes, box_confidence, box_class_probs, threshold = score_threshold)
    
    boxes = scale_boxes(boxes, image_shape)

    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes = max_boxes, iou_threshold = iou_threshold)
    
    return scores, boxes, classes

In [ ]:
#loading the model, classes and anchor boxes
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
model_image_size = (608, 608)

yolo_model = load_model("model_data/", compile=False)
yolo_model.trainable = False

In [ ]:
#model details
yolo_model.summary()

In [ ]:
#predict on the image, process the output, draw the filtred boxes on the image and saving the results to out file
def predict(image_file, confidence_threshold ):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,15))

    # Preprocess the image
    image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))
    axes[0].imshow(image)
    yolo_model_outputs = yolo_model(image_data)
    yolo_outputs = yolo_head(yolo_model_outputs, anchors, len(class_names))
    
    out_scores, out_boxes, out_classes = yolo_eval(yolo_outputs, [image.size[1],  image.size[0]], 10, confidence_threshold, 0.5)
    # Print predictions info
    print('Found {} boxes for {}'.format(len(out_boxes), "images/" + image_file))
    # Generate colors for drawing bounding boxes.
    colors = get_colors_for_classes(len(class_names))
    # Draw bounding boxes on the image file
    #draw_boxes2(image, out_scores, out_boxes, out_classes, class_names, colors, image_shape)
    draw_boxes(image, out_boxes, out_classes, class_names, out_scores)
    # Save the predicted bounding box on the image
    image.save(os.path.join("out", image_file), quality=100)
    plt.figure(2)
    axes[1].imshow(image)
    # Display the results in the notebook
    output_image = PIL.Image.open(os.path.join("out", image_file))

    return out_scores, out_boxes, out_classes

In [ ]:
#predicting on a test image (change to other images or upload your own test image)

confidence_threshold = .45 #tune this

out_scores, out_boxes, out_classes = predict("test.jpg", confidence_threshold)
#out_scores, out_boxes, out_classes = predict("test2.jpg", confidence_threshold)


In [ ]:
#evaluating inference time
#make sure colab runtime is on GPU mode

image, input = preprocess_image("images/test.jpg" , model_image_size = (608, 608))

T = 0
for _ in range(5):
    t1 = time.time()
    yolo_model(input)
    t2 = time.time()
    T += (t2-t1)
T /= 5
print('YOLO inference time : %f s ===> %f FPS' % (T, 1/T)) 
